In [ ]:
import pymc3 as pm
import numpy as np
import theano.tensor as tt

In [ ]:
N = 100
with pm.Model() as model:
    p = pm.Uniform("freq_cheating", 0, 1)
    
    # testval specifies a starting point for sampling
    # TODO: What happens if you remove testval?
    true_answers = pm.Bernoulli("truths", p, shape=N, testval=np.random.binomial(1, 0.5, N))
print(true_answers.tag.test_value)

In [ ]:
with model:
    first_coin_flips = pm.Bernoulli("first_flips", 0.5, shape=N, testval=np.random.binomial(1, 0.5, N))
    
    # Not everyone will flip a second time, but model a possible realization of second flips
    second_coin_flips = pm.Bernoulli("second_flips", 0.5, shape=N, testval=np.random.binomial(1, 0.5, N))
    
print(first_coin_flips.tag.test_value)

In [ ]:
with model:
    # Elements in val are 1 iff: 1) first toss is heads and student cheated, or 2) first toss tails, second toss heads
    val = first_coin_flips*true_answers + (1 - first_coin_flips)*second_coin_flips
    
    # Use pm.Deterministic to create a deterministic (not stochastic) variable representing how many yeses observed 
    observed_proportion = pm.Deterministic("observed_proportion", tt.sum(val)/float(N))

In [ ]:
# Assume the interviewer gets 35 out of 100 yeses
X = 35

with model:
    observations = pm.Binomial("obs", N, observed_proportion, observed=X)

In [ ]:
with model:
    step = pm.Metropolis(vars=[p])
    trace = pm.sample(40000, step=step)
    burned_trace = trace[15000:]

In [ ]:
figsize(12.5, 3)
p_trace = burned_trace["freq_cheating"][15000:]
plt.hist(p_trace, histtype="stepfilled", normed=True, alpha=0.85, bins=30, 
         label="posterior distribution", color="#348ABD")
plt.vlines([.05, .35], [0, 0], [5, 5], alpha=0.3)
plt.xlim(0, 1)
plt.legend()